In [1]:
import scipp as sc
import scippneutron as scn
from ess import loki, sans
from scippneutron.tof.conversions import elastic_wavelength, elastic_Q, beamline
from scipp.constants import m_n, h

In [2]:
def to_bin_edges(d, dim):
    centers = d.coords[dim].copy()
    del d.coords[dim]
    first = 1.5*centers[dim, 0] - 0.5*centers[dim, 1]
    last = 1.5*centers[dim, -1] - 0.5*centers[dim, -2]
    bulk = 0.5 * (centers[dim, 1:] + centers[dim, :-1])
    edges = sc.concat([first, bulk], dim)
    edges = sc.concat([edges, last], dim)
    d.coords[dim] = edges

In [3]:
def get_wavelength_independent_factor(l2, l_collimation, r1, r2, dr):
      
    inv_l3 = (l_collimation + l2) / (l_collimation * l2)
    
    # Terms in Mildner and Carpenter equation.
    # See https://docs.mantidproject.org/nightly/algorithms/TOFSANSResolutionByPixel-v1.html
    a1 = (r1/l_collimation)*(r1/l_collimation) * 3.0
    a2 = (r2*inv_l3)*(r2*inv_l3) * 3.0
    a3 = (dr/l2) * (dr/l2) 
    
    wav_independent = (4.0 * sc.constants.pi / 12) * (a1 + a2 + a3) 
    return wav_independent, a1, a2, a3
    

In [4]:
def generate_gaussian(sigma, term_name, num_points):
    """
    Generates gaussian function based on supplied sigma
    """
    import scipy.stats as stats
    xpoints = sc.linspace('x', -3*sigma, 3*sigma, num=num_points)
    pdf_result = stats.norm.pdf(xpoints.values,loc=0,scale=sigma.value)
    gauss_data = sc.Dataset(
    data={
        term_name:sc.array(dims=['x'], values=pdf_result)}, 
    coords={
        'x':sc.array(dims=['x'], values=xpoints.values)} )
    return gauss_data

In [5]:
def get_sigma_q_value(data, d_lam, lam, wav_independent, moderatorValue, q, l1, l2):

     
    #sigModerator = Moderator time spread (microseconds) as afunction of wavelength (Angstroms)
    sig_moderator = moderatorValue * 3.9560 / (1000.0 * (l1 + l2));
    sig_moderator.unit = sc.units.angstrom
    
    #TODO: replace with q from cooridnate transformations 
    theta = scn.two_theta(data) 
    q_sq = 4.0 * sc.constants.pi * sc.sin(0.5*theta) * sc.reciprocal(lam)
    q_sq *= q_sq
    
    std_dev_lam_sq = q_sq 
    std_dev_lam_sq *= (d_lam * d_lam)/12 + sig_moderator * sig_moderator
    std_dev_lam_sq *= sc.reciprocal(lam * lam)
    
    std_dev_lam_independent = sc.DataArray((wav_independent * sc.reciprocal(lam * lam) ),
                    coords={'wavelength':std_dev_lam_sq.coords['wavelength']}) 
   
    dq_sq = sc.sqrt(std_dev_lam_independent + std_dev_lam_sq)
    return dq_sq   


In [6]:
def q_resolution(wavelength_bins, moderator, data, q):
    
    l_collimation = 4.0*sc.units.m
    r2 = 0.004*sc.units.m
    r1 = 0.01*sc.units.m
    dr = 0.008*sc.units.m
    
    d_lam = wavelength_bins['wavelength', 1:] - wavelength_bins['wavelength', :-1] # bin widths
    lam = sc.midpoints(wavelength_bins)
    l2 = scn.L2(data)
    l1 = scn.L1(data)
    
    wav_independent, a1, a2, a3 = get_wavelength_independent_factor(l2, l_collimation, r1, r2, dr)
    
    to_bin_edges(moderator, 'wavelength')
    moderatorValue = sc.rebin(moderator, 'wavelength', wavelength_bins)
    
    dq_sq = get_sigma_q_value(data, d_lam, lam, wav_independent, moderatorValue, q, l1, l2)
    
    return a1, a2, a3, dq_sq 

In [7]:
# Estimate qresolution function

wavelength_bins = sc.linspace(dim='wavelength', start=2.0, stop=16.0, num=141, unit='angstrom')

path = 'SANS2D_data'
moderator_file = 'ModeratorStdDev_TS2_SANS_LETexptl_07Aug2015.txt'
moderator = loki.io.load_rkh_wav(filename=f'{path}/{moderator_file}')

#q resolution params
l_collimation = 4.0*sc.units.m
r2 = 0.004*sc.units.m
r1 = 0.01*sc.units.m
dr = 0.008*sc.units.m

# Using only one-fourth of the full spectra 245760 (reserved for first detector)
#spectrum_size =  245760//4

# Sample measurement
#sample = loki.io.load_sans2d(filename=f'{path}/SANS2D00063114.nxs',
#                             spectrum_size=spectrum_size)

#d - data is normalized and reduced data

#dq_sq = q_resolution(wavelength_bins, moderator, sample, l_collimation, r1, r2, dr)

#Dense example data - still needs to make it working for event data 
da = scn.data.tutorial_dense_data()

In [8]:
def two_theta(gravity, wavelength, incident_beam, scattered_beam):
    # Arbitrary internal convention: beam=z, gravity=y
    g = sc.norm(gravity)
    L2 = sc.norm(scattered_beam)
    y = sc.dot(scattered_beam, gravity) / g
    n = sc.cross(incident_beam, gravity)
    n /= sc.norm(n)
    x = sc.dot(scattered_beam, n)
    wavelength = sc.to_unit(wavelength, "m", copy=False)
    drop = g * m_n ** 2 / (2 * h ** 2) * wavelength ** 2 * L2 ** 2
    return sc.asin(sc.sqrt(x ** 2 + (y + drop) ** 2) / L2)

In [9]:
#Conversion of the data 
q_with_gravity = {**beamline(scatter=True),
                  **elastic_Q("tof")}
q_with_gravity["two_theta"] = two_theta
da.coords["gravity"] = sans.i_of_q.gravity_vector()
data_q = da.transform_coords("Q", graph=q_with_gravity)


In [10]:
a1, a2, a3, dq_sq = q_resolution(wavelength_bins, moderator, da, data_q.coords['Q'])

The 'bins' keyword argument and positional syntax for setting bin edges is deprecated. Use, e.g., 'sc.rebin(da, x=x_edges)'. See the documentation for details.


### Plotting individual resolution terms 

In [11]:
gauss_data = generate_gaussian(a1, 'a1', 100)

In [12]:
gauss_data['a1'].plot()

### At the end we need to bin to desired q_bins *

In [56]:
dq_sq.coords['sample_position'] = da.coords['sample_position']
dq_sq.coords['source_position'] = da.coords['source_position']
dq_sq.coords['position'] = da.coords['position']
dq_sq.coords["gravity"] = sans.i_of_q.gravity_vector()

In [57]:
dqq = dq_sq.transform_coords("Q", graph=q_with_gravity)

In [60]:
sc.max(dqq.coords['Q'])

<scipp.Variable> ()    float64           [1/Å]  [0.538358]

In [61]:
sc.binning.make_binned(dqq, edges=[q_bins])

BinnedDataError: Binning is only implemented for 1-dimensional data. Consider using groupby, it might be able to do what you need.

In [41]:
q_bins = sc.linspace(dim='Q', start=0.01, stop=0.6, num=141, unit='1/angstrom')
wavelength_bins = sc.linspace(dim='wavelength', start=2.0, stop=16.0, num=141, unit='angstrom')
wavelength_bands = sc.concat(
            [wavelength_bins.min(), wavelength_bins.max()], dim='wavelength')

### Note: a2 and a3 have "spectrum" dimension - not a single value to be plotted